# Install Packages

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
!pip install --upgrade huggingface_hub

In [ ]:
pip install pandas

# Login to Huggingface

In [ ]:
!huggingface-cli login --token "XXX"

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Imports

In [ ]:
from langchain.llms import HuggingFacePipeline

In [ ]:
import transformers
import torch

In [ ]:
from transformers import AutoTokenizer

In [ ]:
import pandas as pd
df = pd.read_csv('/content/LLMs.csv')

In [ ]:
df.head()

# Load llama2 model

In [ ]:
model="meta-llama/Llama-2-7b-chat-hf"
tokenizer=AutoTokenizer.from_pretrained(model)

In [ ]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [ ]:
prompt="Tell me a quote from James Clear?"

In [ ]:
print(llm(prompt))



James Clear is an American author, speaker, and entrepreneur who is known for his work on habits, decision-making, and personal development. Here is a quote from James Clear:

"The only way to get better at something is to do it consistently, over time, and to get feedback on how you're doing. There's no shortcut to mastery, no matter how much you want it."

This quote highlights the importance of consistent effort and feedback in improving one's skills and abilities. It emphasizes that there is no quick fix or shortcut to mastery, and that it takes time, dedication, and hard work to achieve excellence.


# LLMChain

Simple but really powerful chain that underpins a lot of the chains
Import the prompt template. Import the LLM chain.
Initialize the language model that we want to use. Initialize a prompt. And
this prompt is going to take in a variable called BookName. It's going
to ask the LLM to provide a short summary of the book. And then finally, we're going to combine these two things into a chain. And so, this is what we call an LLM chain.
It's just the combination of the LLM and the prompt. But now this chain will let us run through the prompt and into the LLM in a sequential manner.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
template_string = "Provide a two sentence summary of the book called {BookName}?"

In [ ]:
prompt_template=PromptTemplate(input_variables=["BookName"],
                               template=template_string)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)

In [ ]:
BookName = "Atomic Habits"

In [ ]:
response = chain.run(BookName)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
Provide a two sentence summary of the book called Atomic Habits?

> Finished chain.


Atomic Habits is a book by James Clear that provides practical strategies for building good habits and breaking bad ones. The book offers a step-by-step guide on how to create good habits, break bad ones, and make lasting changes in your life.


# SimpleSequential Chain

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
first_template_string = "Who's the author of the book called {BookName}?"
# prompt template 1
first_prompt_template=PromptTemplate(input_variables=["BookName"],
                               template=first_template_string)
#Chain one
chain_one = LLMChain(llm=llm, prompt=first_prompt_template, verbose=True)

In [ ]:
second_template_string = "Name one more book from the Author:{Author}?"
# prompt template 2
second_prompt_template=PromptTemplate(input_variables=["Author"],
                               template=second_template_string)
#Chain two
chain_two = LLMChain(llm=llm, prompt=second_prompt_template, verbose=True)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
BookName = "A tale of two cities"

In [ ]:
overall_simple_chain.run(BookName)

# SequentialChain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
first_template_string = "Translate the following review to english: \n{Review}"
# prompt template 1
first_prompt_template=PromptTemplate(input_variables=["Review"],
                               template=first_template_string)
#Chain one
chain_one = LLMChain(llm=llm, prompt=first_prompt_template,
                     output_key="English_Review", verbose=True)

In [ ]:
second_template_string = "Can you summarize the following review in 1 sentence:\n{English_Review}"
# prompt template 2
second_prompt_template=PromptTemplate(input_variables=["English_Review"],
                               template=second_template_string)
#Chain two
chain_two = LLMChain(llm=llm, prompt=second_prompt_template,
                     output_key="summary", verbose=True)

In [ ]:
third_template_string = "What language is the following review:\n\n{Review}"
# prompt template 3
third_prompt_template=PromptTemplate(input_variables=["Review"],
                               template=third_template_string)
#Chain three
chain_three = LLMChain(llm=llm, prompt=third_prompt_template,
                     output_key="language", verbose=True)

In [ ]:
fourth_template_string = "Write a follow up response to the following summary in the specified language: \nSummary: {summary}\n\nLanguage: {language}"
# prompt template 4
fourth_prompt_template=PromptTemplate(input_variables=["summary", "language"],
                               template=fourth_template_string)
#Chain Four
chain_four = LLMChain(llm=llm, prompt=fourth_prompt_template,
                     output_key="followup_message", verbose=True)

In [ ]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [ ]:
review = "Anteriormente escribí esta crítica justo después de leer el libro. Hoy, 15 de febrero, después de aplicar el sistema de James durante 100 días en unos pocos hábitos diminutos, me siento obligado a compartir actualizaciones con ustedes porque han funcionado sinceramente.Dividiré la revisión en 5 partes. La primera parte es un resumen del libro con breves extractos resaltados mientras toma notas. A continuación, espero compartir consejos que me han motivado a la vez que construyen nuevos hábitos. A continuación, voy a compartir cómo implementé los primeros 3 hábitos a lo largo de estos meses. Entonces, algunos pensamientos a los que recomendaría leer el libro."

In [ ]:
overall_chain(review)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Translate the following review to english: 
Anteriormente escribí esta crítica justo después de leer el libro. Hoy, 15 de febrero, después de aplicar el sistema de James durante 100 días en unos pocos hábitos diminutos, me siento obligado a compartir actualizaciones con ustedes porque han funcionado sinceramente.Dividiré la revisión en 5 partes. La primera parte es un resumen del libro con breves extractos resaltados mientras toma notas. A continuación, espero compartir consejos que me han motivado a la vez que construyen nuevos hábitos. A continuación, voy a compartir cómo implementé los primeros 3 hábitos a lo largo de estos meses. Entonces, algunos pensamientos a los que recomendaría leer el libro.

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Can you summarize the following review in 1 sentence:
 Finalmente, conclusiones y recomendaciones.
What does

{'Review': 'Anteriormente escribí esta crítica justo después de leer el libro. Hoy, 15 de febrero, después de aplicar el sistema de James durante 100 días en unos pocos hábitos diminutos, me siento obligado a compartir actualizaciones con ustedes porque han funcionado sinceramente.Dividiré la revisión en 5 partes. La primera parte es un resumen del libro con breves extractos resaltados mientras toma notas. A continuación, espero compartir consejos que me han motivado a la vez que construyen nuevos hábitos. A continuación, voy a compartir cómo implementé los primeros 3 hábitos a lo largo de estos meses. Entonces, algunos pensamientos a los que recomendaría leer el libro.',
 'English_Review': ' Finalmente, conclusiones y recomendaciones.\nWhat does the review say?\nThe reviewer previously wrote the review after reading the book. Today, February 15th, after applying the system of James for 100 days in a few small habits, they feel compelled to share updates with you because they have work